We check the decomposition of the two-qubit ZZ rotation obtained via ZX calculus with $\gamma=3$ (Table IV in in M. Schumann et al "Bridging wire and gate cutting with ZX-calculus" (2025)).

In [10]:
import numpy as np
import scipy
from util import return_ptm

In [11]:
def apply_rzz_theta(op: np.ndarray, params: dict):
    if op.shape[1] != 4:
        raise ValueError("Matrix shape error: the matrix should be 4x4")
    pauli_z = np.array([[1, 0], [0, -1]])
    theta = params["theta"]
    pauli_zz = np.kron(pauli_z, pauli_z)
    rzz = scipy.linalg.expm(-1j*theta*pauli_zz/2)
    return rzz@op@rzz.conj().T

def apply_rz_theta(op: np.ndarray, params: dict):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix should be 2x2")
    pauli_z = np.array([[1, 0], [0, -1]])
    theta = params["theta"]
    rz = scipy.linalg.expm(-1j*theta*pauli_z/2)
    return rz@op@rz.conj().T

In [12]:
num_qubits = 2
m = 1
m_prime = num_qubits - 1
theta = np.pi*6/5

In [13]:
target_ptm_rzz_theta = return_ptm(apply_rzz_theta, 2, params={"theta": theta})

Now we obtain the PTMs for each term in the decomposition
1. $\mathcal{I} \otimes \mathcal{I}$

In [15]:
deco_dict = {}
deco_dict["1"] = {}
deco_dict["1"]["q"] = 1/2*(1 + np.cos(theta))
deco_dict["1"]["ptm"] = np.kron(return_ptm(apply_rz_theta, m, params={"theta": 0.0}), return_ptm(apply_rz_theta, m_prime, params={"theta": 0.0}))

2. $\mathcal{R}_{Z}(\pi) \otimes \mathcal{R}_{Z}(\pi)$

In [16]:
deco_dict["2"] = {}
deco_dict["2"]["q"] = 1/2*(1 - np.cos(theta))
deco_dict["2"]["ptm"] = np.kron(return_ptm(apply_rz_theta, m, params={"theta": np.pi}), return_ptm(apply_rz_theta, m_prime, params={"theta": np.pi}))

3. $\mathcal{R}_{Z} \left( \frac{\pi}{2} \right) \otimes \mathcal{E}_{\mathcal{R}_{ZZ}(\theta)-\ket{\pm i}_a}$

In [18]:
def apply_rzz_theta_ancilla_measurement(op: np.ndarray, params: dict):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    theta = params["theta"]
    ancilla_state = 1/np.sqrt(2)*np.array([[1], [1]])
    rho_plus_i = 1/2*np.array([[1, -1j], [1j, 1]])
    rho_minus_i = 1/2*np.array([[1, 1j], [-1j, 1]])
    rho_ancilla_state = ancilla_state@ancilla_state.conj().T
    op_with_ancilla = np.kron(op, rho_ancilla_state)
    op_with_ancilla_rzz_theta = apply_rzz_theta(op_with_ancilla, {"theta": theta})
    proj_plus_i = np.kron(np.identity(2), rho_plus_i)
    proj_minus_i = np.kron(np.identity(2), rho_minus_i)
    op_proj_plus_i = proj_plus_i@op_with_ancilla_rzz_theta@proj_plus_i
    op_proj_minus_i = proj_minus_i@op_with_ancilla_rzz_theta@proj_minus_i 
    op_plus_trace = np.trace(op_proj_plus_i.reshape(2 , 2, 2, 2), axis1=1, axis2=3)
    op_minus_trace = np.trace(op_proj_minus_i.reshape(2 , 2, 2, 2), axis1=1, axis2=3)
    # op_trace = np.trace(op_with_ancilla_mcz.reshape(2**n , 2, 2**n, 2), axis1=1, axis2=3) # only for testing
    return op_plus_trace - op_minus_trace
    
    

In [19]:
deco_dict["3"] = {}
deco_dict["3"]["q"] = 1/2
deco_dict["3"]["ptm"] = np.kron(return_ptm(apply_rz_theta, m, params={"theta": np.pi/2}), return_ptm(apply_rzz_theta_ancilla_measurement, m_prime, params={"theta": theta}))

4. $\mathcal{R}_{Z} \left(-\frac{\pi}{2} \right) \otimes \mathcal{E}_{\mathcal{R}_{ZZ}(\theta)-\ket{\pm i}_a} $

In [20]:
deco_dict["4"] = {}
deco_dict["4"]["q"] = -1/2
deco_dict["4"]["ptm"] = np.kron(return_ptm(apply_rz_theta, m, params={"theta": -np.pi/2}), return_ptm(apply_rzz_theta_ancilla_measurement, m_prime, params={"theta": theta}))

5. $\mathcal{\overline{E}}_Z \otimes \mathcal{R}_Z(\theta)$

In [22]:
def apply_measure_and_prepare_z(op: np.ndarray, params=None):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix should be 2x2")
    rho_0 = np.array([[1, 0], [0, 0]])
    rho_1 = np.array([[0, 0], [0, 1]])
    return rho_0*np.trace(rho_0@op) - rho_1*np.trace(rho_1@op)
    

In [23]:
deco_dict["5"] = {}
deco_dict["5"]["q"] = 1/2
deco_dict["5"]["ptm"] = np.kron(return_ptm(apply_measure_and_prepare_z, m, params=None), return_ptm(apply_rz_theta, m_prime, params={"theta": theta}))

6. $\mathcal{\overline{E}}_Z \otimes \mathcal{R}_Z(-\theta)$

In [24]:
deco_dict["6"] = {}
deco_dict["6"]["q"] = -1/2
deco_dict["6"]["ptm"] = np.kron(return_ptm(apply_measure_and_prepare_z, m, params=None), return_ptm(apply_rz_theta, m_prime, params={"theta": -theta}))

In [25]:
cut = sum([deco_dict[key]["q"]*deco_dict[key]["ptm"] for key in deco_dict.keys()])

In [26]:
np.max(np.abs(target_ptm_rzz_theta - cut))

np.float64(5.551115123125783e-16)

which shows that the decomposition is correct!